In [1]:
!pip install python-chess
!pip install psutil
!pip install google-cloud-storage

import numpy as np
import pandas as pd
import glob
import os
import chess


In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/gleaming-modem-400906-3780658b0f47.json"
from google.cloud import storage

def download_from_bucket(blob_name, path_to_save_to, bucket_name):
    client = storage.Client()

    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.download_to_filename(path_to_save_to)
    print(f"File {blob_name} downloaded to {path_to_save_to}.")

# Use the function to download your file:
download_from_bucket("final_4M_bits.npy", "/content/final_4M_bits.npy", "chess_data_all")
download_from_bucket("final_4M_labels.npy", "/content/final_4M_labels.npy", "chess_data_all")


File final_4M_bits.npy downloaded to /content/final_4M_bits.npy.
File final_4M_labels.npy downloaded to /content/final_4M_labels.npy.


In [2]:
bitboard_dataset=np.load("/content/final_4M_bits.npy")
labels=np.load("/content/final_4M_labels.npy")

In [3]:
bitboard_dataset2=bitboard_dataset[:1000000]
labels2=labels[:1000000]


In [4]:
# print(bitboard_dataset2.shape ,bitboard_dataset2[0])
# print(labels2.shape ,labels2[0])

In [5]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, Conv3D, Flatten, Dense, Concatenate
from tensorflow.keras.activations import relu
import tensorflow as tf
from tensorflow.keras.layers import DepthwiseConv2D

bitboard_dataset = bitboard_dataset.transpose(0, 2, 3, 1)
# ... Assuming bitboard_dataset and labels are defined ...

bitboard_dataset, labels = shuffle(bitboard_dataset, labels)
X_train, X_test, y_train, y_test = train_test_split(bitboard_dataset, labels, test_size=0.1)


# Input layer
input_layer_2d = layers.Input(shape=(8, 8, 16))  # for 2D convolutions


In [ ]:
from tensorflow.keras.layers import BatchNormalization, Dropout, Activation

# # Branch 1: Depthwise Convolution
# branch1 = DepthwiseConv2D(kernel_size=(1, 1), depth_multiplier=1, activation=relu)(input_layer_2d)
# branch1 = BatchNormalization()(branch1)


#branch 4 : conv2D(branch1)
branch4 = Conv2D(filters=20, kernel_size=(1, 1), activation=relu)(input_layer_2d)
branch4 = Conv2D(filters=10, kernel_size=(1, 1), activation=relu)(branch4)
branch4 = BatchNormalization()(branch4)


# Branch 3: Conv3 (Only to the 15th channel)
branch3_input = tf.gather(input_layer_2d, [13,14], axis=3)  # Extract the 15th channel
branch3 = Conv2D(filters=20, kernel_size=(1,1), activation=relu)(branch3_input)
branch3 = Conv2D(filters=1, kernel_size=(1, 1), activation=relu)(branch3)
branch3 = Conv2D(filters=1, kernel_size=(8, 8), activation=relu)(branch3)
branch3 = BatchNormalization()(branch3)

# branch1 = Flatten()(branch1)
branch4 = Flatten()(branch4)
branch3 = Flatten()(branch3)

concat = Concatenate()([branch4, branch3])

dense1 = Dense(2000)(concat)
dense1 = BatchNormalization()(dense1)
dense1 = Activation('relu')(dense1)
dense1 = Dropout(0.25)(dense1)

dense2 = Dense(1000)(dense1)
dense2 = BatchNormalization()(dense2)
dense2 = Activation('relu')(dense2)
dense2 = Dropout(0.25)(dense2)

dense3 = Dense(500)(dense2)
dense3 = BatchNormalization()(dense3)
dense3 = Activation('relu')(dense3)
dense3 = Dropout(0.25)(dense3)

dense4 = Dense(200)(dense3)
dense4 = BatchNormalization()(dense4)
dense4 = Activation('relu')(dense4)
dense4 = Dropout(0.25)(dense4)

output_layer = Dense(1)(dense4)

# Create the model
model = Model(inputs=input_layer_2d, outputs=output_layer)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=50, validation_split=0.1)

# Evaluate the model
loss, mae= model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Mean Absolute Error:", mae)

Epoch 1/200
63771/64024 [============================>.] - ETA: 2s - loss: 8442.0811 - mae: 38.3354

In [ ]:
model.save("model.h5")
